In [1]:
TTL_HEADER = """                                                                                                             
@base <https://github.com/Coleridge-Initiative/adrf-onto/wiki/Vocabulary> .                                                  
@prefix cito:   <http://purl.org/spar/cito/> .                                                                               
@prefix dct:    <http://purl.org/dc/terms/> .                                                                                
@prefix foaf:   <http://xmlns.com/foaf/0.1/> .                                                                               
@prefix rdf:    <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .                                                              
@prefix xsd:    <http://www.w3.org/2001/XMLSchema#> .                                                                        
"""

PUB_TEMPLATE = """                                                                                                           
:{}                                                                                                                          
  rdf:type :ResearchPublication ;                                                                                            
  dct:title "{}"@en ;                                                                                                        
  dct:identifier "{}" ;                                                                                                      
  dct:language "en" ;                                                                                                        
  foaf:page "{}"^^xsd:anyURI ;  
  cito:citesAsDataSource "{}";
  dct:subject "{}" ;                                                                                                      
  .                                                                                                                          
"""
# PUB_TEMPLATE = """                                                                                                           
# :{}                                                                                                                          
#   rdf:type :ResearchPublication ;                                                                                            
#   dct:title "{}"@en ;                                                                                                        
#   dct:identifier "{}" ;                                                                                                       
#   dct:language "en" ;                                                                                                        
#   foaf:page "{}"^^xsd:anyURI ;                                                                                               
#   .                                                                                                                          
# """

read in json

In [3]:
import json
partition_path = "metadata_final/20191021_pub_md.json"

with open(partition_path) as json_file:
    usda_md = json.load(json_file)

subset to test

In [4]:
results = usda_md[0:2]

In [5]:
from rdflib import Graph, plugin
from rdflib.plugin import register, Parser, Serializer
import pyld

# format as JSON_LD                                                                                                      
with open("vocab.json", "r") as f:
    CONTEXT = json.load(f)

frags = [TTL_HEADER]

for entry in results:
    pub_id = entry['pub_id']
    title = entry['title']
    doi = entry['doi']
    dataset = entry['datasets']
    try:
        pub_url = entry['url']
    except:
        pub_url = ''
    try:
        pub_pdf = entry['pdf']
    except:
        pub_pdf = ''
    try:
        authors = entry['authors']
    except:
        authors = ''
    try:
        keywords = entry['new_keywords']
    except:
        keywords = ''
    frags.append(PUB_TEMPLATE.format(pub_id, title, doi, pub_url,dataset,keywords))

g = Graph()
g.parse(data="\n".join(frags), format="n3")

jsonld = json.loads(g.serialize(format="json-ld", context=CONTEXT))
jsonld = pyld.jsonld.compact(jsonld, CONTEXT)

print(json.dumps(jsonld, indent=2))

{
  "@context": {
    "@language": "en",
    "adrf": "https://github.com/Coleridge-Initiative/adrf-onto/wiki/Vocabulary#",
    "cito": "http://purl.org/spar/cito/",
    "dct": "http://purl.org/dc/terms/",
    "foaf": "http://xmlns.com/foaf/0.1/",
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "xsd": "http://www.w3.org/2001/XMLSchema#"
  },
  "@id": "adrf:publication-9d21370465e28c315dd9",
  "@type": "adrf:ResearchPublication",
  "dct:identifier": [
    {
      "@value": "10.1257/pandp.20181106"
    },
    {
      "@value": "10.1093/aepp/ppy004"
    }
  ],
  "dct:language": {
    "@value": "en"
  },
  "dct:subject": [
    {
      "@value": "contribution,disadvantaged groups,analysis,United States,importance,immigrant groups,differential incidence,status,food insecurity,differences,recession,group,decomposition analysis,insecurity,ethnicity,structural factors,Great Recession,patterns,immigrant status,factors,race/ethnicity,Racial,state,intensive margin"
    },
    {
      

In [ ]:
# [
#     "10.1111/agec.12444",
#     "publication-e84ab84d25278051773e",
#     "http://purl.umn.edu/205424",
#     "Agricultural Economics",
#     "Does a nutritious diet cost more in food deserts?"
#   ]